<a href="https://colab.research.google.com/github/yesoly/Signature_Recognition/blob/master/signature_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import re
import glob
from PIL import Image
import os, re

In [0]:
# 파일 경로 지정하기
images = glob.glob('/Level3/*.bmp') # 파일 경로

# 전처리

파이썬을 이용한 머신러닝, 딥러닝 실전 개발 입문 > https://wikibook.co.kr/python-machine-learning/ [예제 코드:ch7/avhash.py 참고]

---



In [0]:
def average_hash(fname, size = 100):
    img = Image.open(fname) # 이미지 데이터 열기
    img = img.convert('L') # 그레이스케일로 변환하기
    img = img.resize((size, size), Image.ANTIALIAS) # 56 * 28 리사이즈
    pixel_data = img.getdata() # 픽셀 데이터 가져오기
    pixels = np.array(pixel_data) # Numpy 배열로 변환하기
    pixels = pixels.reshape((size, size)) # 2차원 배열로 변환하기
    avg = pixels.mean() # 평균 구하기
    diff = 1 * (pixels > avg) # 평균보다 크면 1, 작으면 0으로 변환하기
    return diff
    
# 이진 해시로 변환하기
def np2hash(ahash):
    bhash = []
    for nl in ahash.tolist():
        sl = [str(i) for i in nl]
        s2 = "".join(sl)
        i = int(s2, 2) # 이진수를 정수로 변환하기
        bhash.append("%04x" % i)
    return "".join(bhash)

In [0]:
# 해밍 거리 구하기 --- (※2)
def hamming_dist(a, b):
    aa = a.reshape(1, -1) # 1차원 배열로 변환하기
    ab = b.reshape(1, -1)
    dist = (aa != ab).sum()
    return dist

# 이미지 찾기 --- (※4)
def find_image(fname, rate):
  src = average_hash(fname)
  for dst_fname in images:
    dst = average_hash(dst_fname)
    diff_r = hamming_dist(src, dst) / 10000
    # print("[check] ",fname)
    if diff_r > rate:
      print(fname, dst_fname)

# 로지스틱 회귀로 학습

In [0]:
# 학습을 위한 라벨 값 설정 및 해시코드 설정
class User:
  def __init__(self, num, label, name, level,b_hash):
    self.num = num
    self.label = label
    self.name = name
    self.level = level
    self.b_hash = b_hash

img_hash_list = []
img_label_list = []
user_list = []

for fname in images:
  bhash = average_hash(fname)
  img_hash_list.append(bhash.reshape(10000))
  #print(np2hash(ahash))

  label_temp = fname.split('L')[2]
  num = int(label_temp.split('-')[0])

  if (num < 160): # Level1
    user = User(num, 0.0, '나영수', 1, bhash)
    img_label_list.append(0)
  elif (num > 160 and num < 200):
    user = User(num, 1.0, '윤경식', 1, bhash)
    img_label_list.append(1)
  elif (num > 200 and num < 240):
    user = User(num, 0.0, '이서현', 2, bhash)
    img_label_list.append(0) 
  elif(num > 240 and num < 300):
    user = User(num, 1.0, '임민정', 2, bhash)
    img_label_list.append(1)
  elif(num > 300 and num < 320):
    user = User(num, 0.0, '정소정', 3, bhash)
    img_label_list.append(0)
  elif(num > 320 and num < 340):
    user = User(num, 1.0, '박예솔', 3, bhash)
    img_label_list.append(1)

  user_list.append(user)
  

In [0]:
size_row    = 100   # height of the image
size_col    = 100   # width of the image

data = img_hash_list
num_image = len(data)
print(num_image)

for i in range (len(data)) :
  plt.imshow(data[i].reshape(size_row, size_col), cmap='Greys', interpolation='None')
  plt.show()

데이터 학습

In [0]:
def sigmoid_func(z):
  return 1.0 / (1.0 + np.exp(-z))

def sigmoid_grad(z):
  sigmoid =  1.0 / (1.0 + np.exp(-z))
  return sigmoid * (1 - sigmoid)

def cost(y_enc, predict):
  return np.sum(-y_enc * (np.log(predict)) - (1 - y_enc)* np.log(1 - predict)) / len(predict)

def random_theta():
  theta_u = np.random.randn(1000, 10000)
  theta_v = np.random.randn(100, 1000)
  theta_w = np.random.randn(1, 100)
  return theta_u, theta_v, theta_w

def func_calculate(image, u, v, w):
  x = image
  y_ = u.dot(x.T)
  y = sigmoid_func(y_)

  z_ = v.dot(y)
  z = sigmoid_func(z_)

  h_ = w.dot(z)
  h = sigmoid_func(h_)

  return x, y, y_, z, z_, h, h_

def gradient(x, y, z, h, y_, z_, h_, y_enc, u, v, w):
  delta_h = h - y_enc
  delta_z = w.T.dot(delta_h) * sigmoid_grad(z_)

  delta_y = v.T.dot(delta_z) * sigmoid_grad(y_)

  grad_u = delta_y.dot(x)
  grad_v = delta_z.dot(y.T)
  grad_w = delta_h.dot(z.T)

  return grad_u, grad_v, grad_w

def accuracy_train(image, label, predict):
  match = np.sum(label == predict, axis = 0)
  return match/len(label)

In [0]:
def training(lr, epoch, img_hash, img_label):
  cost_converge_value = 1e-7  # 수렴 
  u, v, w = random_theta()

  # Save data
  train_J_list = []
  train_accuracy_list = []

  while (True) :
    x, y, y_, z, z_, h, h_ = func_calculate(img_hash, u, v, w)
    train_J = cost(img_label, h)
    grad_u, grad_v, grad_w = gradient(x, y, z, h, y_, z_, h_, img_label, u, v, w)
    predict = np.around(h)[0]
    u -= lr * grad_u
    v -= lr * grad_v
    w -= lr * grad_w
    train_accuracy = accuracy_train(img_hash, img_label, predict)

    epoch += 1
    train_J_list.append(train_J)
    train_accuracy_list.append(train_accuracy)

    # Print Log
    if epoch % 1 == 0:
      print('Epoch: {:6d}, train_cost: {:10f}, train_accuracy: {:10f}'.format(epoch, train_J, train_accuracy))
    
    if train_accuracy >= 1.0 :
      print("cost is converged")
      print('Epoch: {:6d}, train_cost: {:10f}, train_accuracy: {:10f}'.format(epoch, train_J, train_accuracy))
      print('img_label >' ,img_label)
      print('predict >' ,predict)
      break    

    if len(train_J_list) > 3:
      if abs(train_J_list[-1] - train_J_list[-2]) < cost_converge_value:
        print("cost is converged")
        print('Epoch: {:6d}, train_cost: {:10f}, train_accuracy: {:10f}'.format(epoch, train_J, train_accuracy))
        break 

  return train_J_list, train_accuracy_list, h

In [0]:
learning_rate = 0.001
epoch = 0
hash_arr = np.array(img_hash_list)
label_arr = np.array(img_label_list)
print("=========================== Training Start ===========================")
J_list, accuracy_list, img_predict = training(learning_rate, epoch, hash_arr, label_arr)

In [0]:
# user(self, num, label, name, level, b_hash):
for index, user in enumerate(user_list):
  if(user.label == img_predict[index]):
    print(user.name)
    plt.imshow(user.b_hash.reshape(size_row, size_col), cmap='Greys', interpolation='None')
    plt.imshow(images[index], cmap='Greys', interpolation='None')

plt.show()

In [0]:
print("=========================== Failed Image ===========================")
find_fail_img(hash_arr, label_arr, img_predict)